## Notebook function:
* Connect to API at US Energy Information Agency and download hourly electric grid data *
https://www.eia.gov

API URL:
https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&start=2015-07-01T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

Method:  GET

Series description:
Hourly demand, day-ahead demand forecast, net generation, and interchange by balancing authority. Source: Form EIA-930 Product: Hourly Electric Grid Monitor

API Documentation:  https://www.eia.gov/opendata/documentation.php

In [50]:
import requests
import json
import pandas as pd
import time

In [54]:
#retrieves data from EIA api, returns json with response, request, and api metadata
def getEIAdata(api_keystring, start, end, offset, length):
    url = (f"https://api.eia.gov/v2/electricity/rto/region-data/data/?{api_keystring}frequency=hourly&data[0]"
       f"=value&start={start}&end={end}&sort[0][column]=period&sort[0][direction]"
       f"=desc&offset={offset}&length={length}")
    try:
        response = requests.get(url)
    except:
        print(f'no response from {url}')
    try:
        response_json = json.dumps(response.json(), indent=4)
    except:
        print(response)
    return response_json

In [41]:
#returns subset of dictionary containing data items
def extractData(response_dict):
    data = response_dict['response']['data']
    return data

In [42]:
#extracts total number of items that matched API request
def extractTotalRows(response_dict):
    total_rows = response_dict['response']['total']
    return total_rows

In [43]:
#appends data to json file
def saveJSON(json_obj, data_file_path):
    with open(data_file_path, 'a') as fout:
        fout.write(json_obj)

In [44]:
#appends data to csv file
def saveCSV(data_dict, csv_path, header):
    df = pd.DataFrame.from_dict(data_dict)
    with open(csv_path, 'a') as fout:
        df.to_csv(fout, header=header, index=False, lineterminator='\n')

In [58]:
#flow
api_key = 'DryvLQciETN0UgsSlqTeeQnSfHj8sPif8tfUGKCg'
api_keystring = f"api_key={api_key}&"
row_limit = 5000
offset = 0
start_date = '2022-01-01'
end_date = '2022-03-27'
start_datetime = f"{start_date}T00" #API takes start and end hour in '2023-04-02T00' format
end_datetime = f"{end_date}T00"

json_path = 'eiadata.json'
csv_path = 'eia.csv'
with open(json_path, 'w') as overwrite:
    pass
with open(csv_path, 'w') as overwrite:
    pass

response_json = getEIAdata(api_keystring, start_datetime, end_datetime, offset, row_limit)
#create dictionary from json object
d = json.loads(response_json)
data = extractData(d)
saveJSON(response_json, json_path)
saveCSV(data, csv_path, header=True)
returned_rows = len(data)
total_rows = extractTotalRows(d)
print(total_rows)
call_count = 1
while call_count * row_limit < total_rows:
    offset = call_count * row_limit
    response_json = getEIAdata(api_keystring, start_datetime, end_datetime, offset, row_limit)        
    d = json.loads(response_json)
    data = extractData(d)
    #only save first and last chunk to json for examination
    #saveJSON(response_json, json_path)
    saveCSV(data, csv_path, header=False)
    call_count += 1
    print(call_count*row_limit)
    if call_count % 20 == 0:
        time.sleep(60)
    if call_count % 200 == 0:
        time.sleep(600)
length = total_rows - (call_count - 1)*row_limit
response_json = getEIAdata(api_keystring, start_datetime, end_datetime, offset+row_limit, length)
d = json.loads(response_json)
data = extractData(d)
saveJSON(response_json, json_path)
saveCSV(data, csv_path, header=False)


586955
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000


In [14]:
#change this to dataframe from csv
df = pd.DataFrame.from_dict(d['response']['data'])
print(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   period           5000 non-null   object
 1   respondent       5000 non-null   object
 2   respondent-name  5000 non-null   object
 3   type             5000 non-null   object
 4   type-name        5000 non-null   object
 5   value            5000 non-null   int64 
 6   value-units      5000 non-null   object
dtypes: int64(1), object(6)
memory usage: 273.6+ KB
